In [1]:
%matplotlib inline

In [2]:
%run ./setup_notebook.ipynb

In [3]:
import pickle
import run_functions as rf
from subsets_analysis import Subsets

In [4]:
run_dict = {
            'J048' : [27, 29, 30, 32], 
            'RL048': [23, 24, 25, 28, 29]
           }

# just take the first session from the first mouse for now
mouse = list(run_dict.keys())[0]
run_number  = run_dict[mouse][0]

pkl_path = '/home/jamesrowland/Documents/Code/Vape/run_pkls/'

run_path = os.path.join(pkl_path, mouse, 'run{}.pkl'.format(run_number))

with open (run_path, 'rb') as f:
    run = pickle.load(f)

In [5]:
# how many frames before and after the onset of lick trial to include in the kernal
pre_frames  = 5
post_frames = 9
fs = 5  # frame rate / plane
offset = 4

In [6]:
# the number of cells stimulated on each trial
trial_subsets = Subsets(run).trial_subsets

In [7]:
run = rf.filter_unhealthy_cells(run)
run = utils.lick_binner(run)

In [8]:
run.__dict__.keys()

dict_keys(['mouse_id', 'df', 'rows_2p', 'rows_1p', 'dates_2p', 'paqs', 'naparm_folders', 'blimp_folders', 'pycontrol_folders', 'prereward_folders', 'tseries_folders', 'plane_numbers', 'run_pycontrol_txt', 'num_planes', 'blimp_path', 'naparm_path', 'pycontrol_path', 'paq_path', 'prereward_path', 'tseries_paths', 'align_barcode', 'trial_info', 'session', 'print_lines', 'trial_time', 'outcome', 'slm_barcode', 'slm_trial_number', 'nogo_barcode', 'nogo_trial_number', 'alltrials_barcodes', 'rsync', 'trial_start', 'paq_rsync', 'frame_clock', 'x_galvo_uncaging', 'slm2packio', 'paq_rate', 'aligner', 'paq_correct', 'pre_rsync', 'pre_licks', 'pre_reward', 'prereward_aligner', 'both_aligner', 'num_frames', 's2p_path', 'flu_raw', 'frames_ms', 'frames_ms_pre', 'flu', 'spks', 'stat', 'licks', 'binned_licks_easytest'])

In [9]:
flu_array = utils.build_flu_array(run, run.trial_start, pre_frames, post_frames)

# remove frames with artifact
artifact_free = np.hstack((
                np.arange(pre_frames), 
                np.arange(pre_frames+offset, pre_frames+post_frames
                         )))

flu_array = flu_array[:, :, artifact_free]



In [52]:
### filter out trials
# remove trials where 150 cells were stimmed (longer and larger artifact)
non_150 = trial_subsets!=150
non_nan = ~np.isnan(np.mean(np.mean(flu_array,0),1))
trials_keep = non_150 & non_nan

In [73]:
class Session():
    def __init__(self):
        self.flu_array = flu_array[:, trials_keep, :]
        self.trial_type = np.array(run.trial_type)[trials_keep]
        self.outcome = run.outcome[trials_keep]
        assert self.flu_array.shape[1] == len(self.trial_type) == len(self.outcome)


In [78]:
session = Session()

In [80]:
with open('session1.pkl', 'wb') as f:
    pickle.dump(Session, f)